In [39]:
import os
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os
import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable, Function
from torch.nn import init
#dtype = torch.cuda.FloatTensor
dtype = torch.FloatTensor

In [40]:
def save_checkpoint(state,filename):
    torch.save(state, filename)  
    print("State Saved")
    print(filename)

In [46]:
areanumbers = [18]#,2,3,3,5,6,7,8,9,10,11,12,13,14,15,16]

batchsize=1
#Train 80% of the dataset 4014
train_test_split =3208
val_amount = 0

def hours_to_datestring(t):
    t=int(t)
    start = datetime.datetime(1950,1,1)
    delta = datetime.timedelta(hours=t)
    return (start+delta).strftime('%Y-%m-%d %H:%M:%S')
# Limit to 2017-12-23 for standardisation
times = np.load('dataset/times.npy')[:365*11]
print(hours_to_datestring(times[0]))
print(hours_to_datestring(times[3292]))

areatemps=[]
for i in areanumbers:
    print(i)
    nextarea = (np.load(f'dataset/by-area/area{i}.npy')[:365*11])
    # Ignore leap days
    for j in range(365):
        days = nextarea[j::365]
        days -= np.mean(days)
        days /= np.std(days)
    areatemps.append(nextarea)
areatemps = np.array(areatemps)
print(nextarea.shape)

def input_indices(end_indices):
    a = end_indices[:,None]
    return np.concatenate((a-4,a-3,a-2,a-1),axis=1)

def new_size(size):
    return torch.Size((size[0]*size[1],))+size[2:]
train_val_beginnings = 4+np.random.permutation(train_test_split-4)
#train_val_beginnings = 4+np.arange(train_test_split-4)

train_inputs = torch.from_numpy(areatemps[:,input_indices(train_val_beginnings[val_amount:])]).type(dtype)
train_inputs = train_inputs.view(new_size(train_inputs.size()))
print(train_inputs.shape)

train_ends = torch.from_numpy(areatemps[:,train_val_beginnings[val_amount:]]).type(dtype)
train_ends = train_ends.view(new_size(train_ends.size()))
print(train_ends.shape)

train_data = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(train_inputs,train_ends),
                                         batch_size=batchsize, shuffle=True
                                        )

2006-12-27 12:00:00
2016-01-01 12:00:00
18
(4015, 64, 64)
torch.Size([3204, 4, 64, 64])
torch.Size([3204, 64, 64])


In [42]:
def k(distsq,D,dt):
    return torch.exp(-distsq/(4*D*dt))/(4*np.pi*D*dt)
def gradientk(dist,D,dt):
    return dist*torch.exp(-(dist**2).sum(1)/(4*D*dt))/(8*np.pi*D**2*dt**2)
class GaussianConvolution(Function):
    D = 0.45
    dt = 1
    @staticmethod
    def forward(ctx, w, I):
        ctx.save_for_backward(w,I)
        interval=torch.arange(I.size()[-1]).type(dtype)
        x1 = interval[None,:,None,None,None]
        x2 = interval[None,None,:,None,None]
        y1 = interval[None,None,None,:,None]
        y2 = interval[None,None,None,None,:]
        distsq = (x1-y1-w[:,0,:,:,None,None])**2+(x2-y2-w[:,1,:,:,None,None])**2
        return (I[:,None,None,:,:]*k(distsq,GaussianConvolution.D,GaussianConvolution.dt)).sum(4).sum(3)
    
    @staticmethod
    def backward(ctx, grad_output):
        w,I = ctx.saved_variables
        w=w.data
        I=I.data
        interval=torch.arange(I.size()[-1]).type(dtype)
        x1 = interval[None,:,None,None,None]
        x2 = interval[None,None,:,None,None]
        y1 = interval[None,None,None,:,None]
        y2 = interval[None,None,None,None,:]
        distx = (x1-w[:,0,:,:,None,None]-y1)[:,None,:,:,:,:].repeat(1,1,1,1,1,I.size()[-1])
        disty = (x2-w[:,1,:,:,None,None]-y2)[:,None,:,:,:,:].repeat(1,1,1,1,I.size()[-1],1)
        dist = torch.cat((distx,disty),dim=1)
        grad = Variable((I[:,None,None,:,:]*gradientk(dist,GaussianConvolution.D,GaussianConvolution.dt)).sum(5).sum(4), requires_grad=False)
        #I(x) only depends on w(x) and not on w(z) for z != x
        return grad*grad_output[:,None,:,:], None

In [43]:
def initialize_weights(*models):
    for model in models:
        for module in model.modules():
            if isinstance(module, nn.Conv2d) or isinstance(module, nn.ConvTranspose2d) or isinstance(module, nn.Linear):
                #nn.init.kaiming_normal(module.weight)
                nn.init.xavier_normal(module.weight,gain=1)
                if module.bias is not None:
                    module.bias.data.zero_()
            elif isinstance(module, nn.BatchNorm2d):
                module.weight.data.fill_(1)
                module.bias.data.zero_()
class _EncoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(_EncoderBlock, self).__init__()
        self.cv1 = nn.Conv2d(in_channels, in_channels, kernel_size=3,padding=1)
        self.bn1 =  nn.BatchNorm2d(in_channels)
        self.lr1 =  nn.LeakyReLU(0.1,inplace=True)
        self.cv2 = nn.Conv2d(in_channels, out_channels, kernel_size=3)
        self.bn2 =  nn.BatchNorm2d(out_channels)
        self.lr2 =  nn.LeakyReLU(0.1,inplace=True)
        self.maxp = nn.MaxPool2d(kernel_size=3, stride=1)
        
    def forward(self, x):
        residual=x
        #print("residual",residual.shape)
        out=self.cv1(x)
        #print("cv1",out.shape)
        out=self.bn1(out)
        #print("bn1",out.shape)
        out=self.lr1(out)
        #print("lr1",out.shape)
        out+=residual
        out=self.cv2(out)
        out=self.bn2(out)
        out=self.lr2(out)
        out=self.maxp(out)
        return out

class _DecoderBlock(nn.Module):
    def __init__(self, in_channels, middle_channels, out_channels):
        super(_DecoderBlock, self).__init__()
        self.cv1 =     nn.Conv2d(in_channels, in_channels, kernel_size=3,padding=1)
        self.bn1 =     nn.BatchNorm2d(in_channels)
        self.lr1 =     nn.LeakyReLU(0.1,inplace=True)
        self.cv2 =     nn.Conv2d(in_channels, middle_channels, kernel_size=3)
        self.bn2 =     nn.BatchNorm2d(middle_channels)
        self.lr2 =     nn.LeakyReLU(0.1,inplace=True)
        self.tcv =     nn.ConvTranspose2d(middle_channels, out_channels, kernel_size=3)
        
    def forward(self, x):
        residual=x
        #print("residual",residual.shape)
        out=self.cv1(x)
        #print("cv1",out.shape)
        out=self.bn1(out)
        #print("bn1",out.shape)
        out=self.lr1(out)
        #print("lr1",out.shape)
        out+=residual
        out=self.cv2(out)
        out=self.bn2(out)
        out=self.lr2(out)
        out=self.tcv(out)
        return out

class _CenterBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(_CenterBlock, self).__init__()
        self.cv1 = nn.Conv2d(in_channels,in_channels , kernel_size=3,padding=1)
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.lr1 = nn.LeakyReLU(0.1,inplace=True)
        self.cv2 = nn.Conv2d(in_channels, out_channels, kernel_size=3)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.lr2 = nn.LeakyReLU(0.1,inplace=True)
        self.tcv = nn.ConvTranspose2d(out_channels, out_channels, kernel_size=3)
        
    def forward(self, x):
        residual=x
        out=self.cv1(x)
        out=self.bn1(out)
        out=self.lr1(out)
        out+=residual
        out=self.cv2(out)
        out=self.bn2(out)
        out=self.lr2(out)
        out=self.tcv(out)
        return out

class CDNN(nn.Module):
    def __init__(self,k):
        super(CDNN, self).__init__()
        self.enc1 = _EncoderBlock(k,64)
        self.enc2 = _EncoderBlock(64, 128)
        self.enc3 = _EncoderBlock(128, 256)
        self.enc4 = _EncoderBlock(256, 512)
        self.dec4= _CenterBlock(512, 386)
        self.dec3 = _DecoderBlock(386+256, 256, 194)
        self.dec2 = _DecoderBlock(194+128, 128, 98)
        self.dec1 = _DecoderBlock(98+64, 64, 2)
        self.final = nn.Sequential(
            nn.Conv2d(2,2, kernel_size=3),
        )
        initialize_weights(self)

    def forward(self, x):
        enc1 = self.enc1(x)
        #print("x",x.shape)
        #print("enc1",enc1.shape)
        enc2 = self.enc2(enc1)
        #print("enc2",enc2.shape)
        enc3 = self.enc3(enc2)
        #print("enc3",enc3.shape)
        enc4 = self.enc4(enc3)
        #print("enc4",enc4.shape)
        dec4 = self.dec4(enc4)
        #print("dec4",dec4.shape)
        dec3 = self.dec3(torch.cat([dec4, F.upsample(enc3, dec4.size()[2:], mode='bilinear')], 1))
        #print("dec3",dec3.shape)
        dec2 = self.dec2(torch.cat([dec3, F.upsample(enc2, dec3.size()[2:], mode='bilinear')], 1))
        #print("dec2",dec2.shape)
        dec1 = self.dec1(torch.cat([dec2, F.upsample(enc1, dec2.size()[2:], mode='bilinear')], 1))
        final = self.final(dec1)
        W=F.upsample(final, x.size()[2:], mode='bilinear')
        #print("W",W.shape)

        return W

In [44]:
net = CDNN(4)

warping = GaussianConvolution.apply
loss_fn = torch.nn.MSELoss(size_average=False)

In [47]:
learning_rate=1e-3
Iter_Size=100
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
losses = []
for epoch in range(20,100):
    print("epoch = ",epoch)
    step=0
    while step< len(train_data)-Iter_Size:
        optimizer.zero_grad()
        iter_loss=0
        i=0
        while(i<Iter_Size):
            Input, Target = next(iter(train_data))
            i=i+1
            step=step+1
            Input = Variable(Input,requires_grad=False)
            Target = Variable(Target)
            w = net(Input)
            y_pred = warping(w, Input[:,-1])
                
            loss = loss_fn(y_pred, Target)/Iter_Size
            loss.backward()
            iter_loss+=loss.data[0]
        print(step,iter_loss)
        losses.append(iter_loss)
        optimizer.step()
    filename="models/MSE/CDNN_MSE_Epoch={}.pth".format(epoch)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': net.state_dict(),
        'optimizer' : optimizer.state_dict(),
        'losses'    : losses,
    },filename)    

epoch =  20
100 59.462466806173325
200 134.11078596115112
300 62.717631101608276
400 61.747832506895065
500 55.093587920069695
600 49.72612227499485
700 47.63769246637821
800 50.79288873076439
900 45.826813980937004
1000 45.16409333050251
1100 48.38779094815254
1200 44.568628653883934
1300 49.67068001627922
1400 51.980880841612816
1500 49.14210547506809
1600 46.28816843032837
1700 43.422235161066055
1800 44.86319401860237
1900 41.73106178641319
2000 47.54289948940277
2100 45.31456145644188
2200 43.10187664628029
2300 45.355447202920914
2400 44.21266309916973
2500 43.87958471477032
2600 40.60140861570835
2700 46.41489861905575
2800 45.700280755758286
2900 43.68738594651222
3000 40.04678601026535
3100 45.36666922271252
3200 43.01696968078613
State Saved
models/MSE/CDNN_MSE_Epoch=20.pth
epoch =  21
100 43.07311537861824
200 42.54684357345104
300 41.1342806071043
400 38.67177537083626
500 39.03800444304943
600 41.8646254837513
700 38.990924805402756
800 43.20360590517521
900 41.11201062798

KeyboardInterrupt: 

In [ ]:
plt.figure()
plt.plot(losses)
plt.xlabel('step')
plt.ylabel('Loss')
plt.suptitle("MSE Loss for Area {0}".format(areanumbers[0]))
plt.show()

In [ ]:
plt.figure()
plt.imshow(train_ends[-1].cpu().numpy(),origin='lower')
plt.show()
plt.figure()
plt.imshow(y_pred.data[0].cpu().numpy(),origin='lower')
plt.show()

In [ ]:
#net.training == True
#.eval()
#.train()

# for loading
#net2=CDNN(4)
#net2.load_state_dict(torch.load(filename))
#net2.eval()

#checkpoint = torch.load(filename)
#start_epoch = checkpoint['epoch']
#net2.load_state_dict(checkpoint['state_dict'])
#optimizer.load_state_dict(checkpoint['optimizer'])
#L=checkpoint['losses']